## Imports

In [114]:
# System functionality
import sys
sys.path.append('..')

import glob
import os
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import graphviz
import math as math
import json
import cv2
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sympy import *
from skimage import data, io, filters
from numpy import linspace
from numpy.core.umath import pi
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from emoji_extractor.extract import Extractor
from emojipedia import Emojipedia
from skimage import io
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
#from utils.plotter import plot_SVM_DecisionBoundary
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, log_loss
from pandas.api.types import CategoricalDtype
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

## Load csvs

In [83]:
path_counts = '/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/random_per_user_emoji_counts.csv'
path_users = '/home/stelios/Desktop/Honours Project/Samples/user_data/ran.csv'
counts_df = pd.read_csv(path_counts,index_col=None, header=0, low_memory = False)
users_df = pd.read_csv(path_users,index_col=None, header=0, low_memory = False)

## % Feuture representation

In [84]:
# Feature % representation
for index, row in counts_df.iterrows():
    counts_df.loc[index] = counts_df.loc[index]/counts_df.loc[index].sum()

In [89]:
# Add gender column
gender_counts_df = counts_df 
gender_counts_df['gender'] = users_df['gender']
gender_counts_df = gender_counts_df.dropna()
gender_counts_df.describe()

,👨🏿‍🔬,🚵🏽‍♂️,👨🏾‍🎤,👩🏾‍🏫,🙍🏽‍♂️,🇮🇨,👳🏾‍♀️,👨🏾‍🏫,🦠,💂🏻‍♀️,...,🇸🇨,🇦🇼,🇰🇾,🇰🇬,🇼🇸,🇦🇶,🇱🇦,🇲🇪,🇲🇵,🇧🇼
count,3.062000e+03,3062.0,3062.0,3062.0,3.062000e+03,3.062000e+03,3.062000e+03,3062.0,3.062000e+03,3062.0,...,3062.0,3062.0,3.062000e+03,3062.0,3062.000000,3.062000e+03,3062.0,3.062000e+03,3062.0,3062.000000
mean,2.521884e-07,0.0,0.0,0.0,6.557910e-07,6.347598e-07,7.850575e-07,0.0,6.280460e-07,0.0,...,0.0,0.0,8.685743e-07,0.0,0.000005,9.583672e-07,0.0,9.009212e-07,0.0,0.000032
std,1.395493e-05,0.0,0.0,0.0,3.628842e-05,3.512465e-05,4.344143e-05,0.0,3.475314e-05,0.0,...,0.0,0.0,4.806285e-05,0.0,0.000301,3.756020e-05,0.0,4.985278e-05,0.0,0.001749
min,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.000000e+00,0.0,0.000000,0.000000e+00,0.0,0.000000e+00,0.0,0.000000
25%,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.000000e+00,0.0,0.000000,0.000000e+00,0.0,0.000000e+00,0.0,0.000000
50%,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.000000e+00,0.0,0.000000,0.000000e+00,0.0,0.000000e+00,0.0,0.000000
75%,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,...,0.0,0.0,0.000000e+00,0.0,0.000000,0.000000e+00,0.0,0.000000e+00,0.0,0.000000
max,7.722008e-04,0.0,0.0,0.0,2.008032e-03,1.943635e-03,2.403846e-03,0.0,1.923077e-03,0.0,...,0.0,0.0,2.659574e-03,0.0,0.016667,1.555210e-03,0.0,2.758621e-03,0.0,0.096774


In [100]:
gender_counts_df = gender_counts_df.replace('female', 0)
gender_counts_df = gender_counts_df.replace('male', 1)
gender_counts_df

,👨🏿‍🔬,🚵🏽‍♂️,👨🏾‍🎤,👩🏾‍🏫,🙍🏽‍♂️,🇮🇨,👳🏾‍♀️,👨🏾‍🏫,🦠,💂🏻‍♀️,...,🇦🇼,🇰🇾,🇰🇬,🇼🇸,🇦🇶,🇱🇦,🇲🇪,🇲🇵,🇧🇼,gender
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## Split into test and train

In [103]:
X_train_gender, X_test_gender, y_train_gender, y_test_gender = train_test_split(gender_counts_df.drop(columns=['gender']), gender_counts_df['gender'], test_size=0.2, random_state=42)

## SVM with Chi-squared kernel Classification

In [135]:
svm_chi = SVC(kernel=chi2_kernel, probability=True).fit(X_train_gender, y_train_gender)

# Train Data
pred_prob_chi = svm_chi.predict_proba(X_train_gender)
logloss_train_chi = log_loss(y_train_gender, pred_prob_chi)
predict_train_chi  = svm_chi.predict(X_train_gender)
accuracy_train_chi = accuracy_score(y_train_gender, predict_train_chi)

# Test Data
pred_prob_val_chi = svm_chi.predict_proba(X_test_gender)
logloss_val_chi = log_loss(y_test_gender, pred_prob_val_chi)
predict_val_chi  = svm_chi.predict(X_test_gender)
accuracy_val_chi = accuracy_score(y_test_gender, predict_val_chi)

# Report
print('Log loss with SVM (Chi-squared kernel) on training set: {:.3f}'.format(logloss_train_chi))
print('Log loss with SVM (Chi-squared kernel) on test set: {:.3f}\n'.format(logloss_val_chi))
print('Accuracy score with SVM (Chi-squared kernel) on training set: {:.3f}'.format(accuracy_train_chi))
print('Accuracy score with SVM (Chi-squared kernel) on test set: {:.3f}'.format(accuracy_val_chi))

Log loss with SVM (Chi-squared kernel) on training set: 0.305
Log loss with SVM (Chi-squared kernel) on test set: 0.549

Accuracy score with SVM (Chi-squared kernel) on training set: 0.913
Accuracy score with SVM (Chi-squared kernel) on test set: 0.693


## Linear SVM Classification

In [137]:
clf = LinearSVC(random_state=0, tol=1e-5).fit(X_train_gender, y_train_gender)

# Train Data
predict_train_clf  = clf.predict(X_train_gender)
accuracy_train_clf = accuracy_score(y_train_gender, predict_train_clf)

# Test Data

predict_val_clf  = clf.predict(X_test_gender)
accuracy_val_clf = accuracy_score(y_test_gender, predict_val_clf)

# Report
print('Accuracy score with Linear SVM on training set: {:.3f}'.format(accuracy_train_clf))
print('Accuracy score with Linear SVM on test set: {:.3f}'.format(accuracy_val_clf))

Accuracy score with Linear SVM on training set: 0.752
Accuracy score with Linear SVM on test set: 0.662
